In [52]:
import spacy
from spacy.lang.en.examples import sentences
import pandas as pd
import numpy as np
import json
import numpy as np
import nltk
from nltk.stem.snowball import SnowballStemmer
from tqdm.notebook import tqdm
from nltk.corpus import stopwords
import os

#nltk.download('stopwords')

In [4]:
my_list = stopwords.words('english')

In [16]:
os.chdir(r"C:\Users\Shipt\Downloads")

df = []

def parse_obj(obj):
    for key in obj:
        if isinstance(obj[key], str):
            obj[key] = obj[key].encode('latin_1').decode('utf-8')
        elif isinstance(obj[key], list):
            obj[key] = list(map(lambda x: x if type(x) != str else x.encode('latin_1').decode('utf-8'), obj[key]))
        pass
    return obj

def load_all_messages(file):
    # Open first the first message
    file = open(file)
    
    #Here we have the decoder from messnenger
    data = json.load(file, object_hook=parse_obj)
    
    df = pd.json_normalize(data['messages'])
    
    return(df)




,sender_name,timestamp_ms,content,type,is_unsent,is_taken_down,bumped_message_metadata.bumped_message,bumped_message_metadata.is_bumped,reactions,photos,audio_files,videos,share.link,sticker.uri,call_duration,gifs,files
0,Josh Shipton,1665554736260,no just come Jake isnt even there,Generic,False,False,no just come Jake isnt even there,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Jack Kempton,1665554279814,Do I have to ask permission for essence?,Generic,False,False,Do I have to ask permission for essence?,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Jack Kempton,1665554271550,Will be back at 4,Generic,False,False,Will be back at 4,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Jack Kempton,1665554264393,Beautiful. Such a nice place,Generic,False,False,Beautiful. Such a nice place,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Jack Kempton,1665554255790,What cunt that’s so whack. Egooooo,Generic,False,False,What cunt that’s so whack. Egooooo,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,Josh Shipton,1648886256969,Did you roll with Lachy,Generic,False,False,Did you roll with Lachy,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9996,Josh Shipton,1648886253808,Yea be said he was gonna,Generic,False,False,Yea be said he was gonna,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9997,Jack Kempton,1648886247245,Cool how Jake came along,Generic,False,False,Cool how Jake came along,False,"[{'reaction': '❤', 'actor': 'Josh Shipton'}]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9998,Josh Shipton,1648886246572,Awesome,Generic,False,False,Awesome,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [100]:
def clean_data(df):
    #We want a usable time stamp
    df['date_time']=pd.to_datetime(df['timestamp_ms'], unit='ms') 
    
    #Way easier to work with lower cases for text
    df['content']=df['bumped_message_metadata.bumped_message'].str.lower()
    
    #only want to work with text, not smart enough to analyse the rest
    try:
        df.drop(columns=['timestamp_ms','gifs','is_unsent','photos','type','videos','audio_files','sticker.uri',
                         'call_duration','share.link','files', 'bumped_message_metadata.is_bumped', 'is_taken_down', 'bumped_message_metadata.bumped_message', 'reactions'],inplace=True)

    #sometimes they dont have files and gifs, if this is the case just tries again without them
    except KeyError:
          df.drop(columns=['timestamp_ms','is_unsent','photos','type','videos','audio_files','sticker.uri',
                         'call_duration', 'bumped_message_metadata.is_bumped', 'is_taken_down', 'bumped_message_metadata.bumped_message', 'reactions'],inplace=True)
        
    #add share link when not working with gc's probs a better way to fix this but ceebs
        
    df['year']=df['date_time'].dt.year
    #df=df[df['year']==2021]
    df['hour']=df['date_time'].dt.hour
    df['weekday']=df['date_time'].dt.weekday
    
    #We can exclude some non participing people
    df=df[~df['sender_name'].isin([''])]
    
    df['content']=df.content.fillna('')
    #df=df[~df['content'].isna()].reset_index()
    
    return (df)


In [125]:
display(df)

,sender_name,content,date_time,year,hour,weekday,share.link,word_number,is_message
0,Josh Shipton,no just come jake isnt even there,2022-10-12 06:05:36.260,2022,6,2,NaN,33,1
1,Jack Kempton,do i have to ask permission for essence?,2022-10-12 05:57:59.814,2022,5,2,NaN,40,1
2,Jack Kempton,will be back at 4,2022-10-12 05:57:51.550,2022,5,2,NaN,17,1
3,Jack Kempton,beautiful. such a nice place,2022-10-12 05:57:44.393,2022,5,2,NaN,28,1
4,Jack Kempton,what cunt that’s so whack. egooooo,2022-10-12 05:57:35.790,2022,5,2,NaN,34,1
...,...,...,...,...,...,...,...,...,...
6369,Jack Kempton,unhappy ian 😂😧,2021-05-30 14:10:16.969,2021,14,6,NaN,14,1
6370,Jack Kempton,was that an esteema lock??,2021-05-30 14:10:01.414,2021,14,6,NaN,26,1
6371,Josh Shipton,,2021-05-30 10:05:53.780,2021,10,6,NaN,0,1
6372,Josh Shipton,l,2021-05-30 09:58:02.153,2021,9,6,NaN,1,1


In [98]:
def add_information( df): 
    #number of word per message
    df['word_number'] = df['content'].apply(lambda x: len(str(x)))
  
    #what else can i add?
    #can add reactions if i really want...
    #can add who unsends the most messages...

    return(df)


In [144]:
def sender_stats(df):
    df['is_message']= 1
    df_grouped=df.groupby('sender_name').agg({'word_number': ['sum', 'max', 'mean', 'median'],})
    
    return(df_grouped)


    # i think this is flipped between am and pm
def time stats(df):
    hour = df.groupby("hour").agg({'word_number': ['sum', 'mean']})
    return hour
    
    

word_number           
             sum       mean
hour                       
0          16041  25.301262
1          26757  26.465875
2          28655  31.733112
3          30411  29.640351
4          45073  28.135456
5          41200  35.273973
6          50404  33.116951
7          35944  29.804312
8          32118  27.928696
9          29582  27.289668
10         36379  30.391813
11         18836  26.908571
12         27867  27.509378
13         34405  29.969512
14         17706  28.329600
15          3255  28.805310
16           146  13.272727
17           127  14.111111
18            59  29.500000
20           186  16.909091
21           144  18.000000
22          1258  40.580645
23          5145  25.725000

In [134]:
def by_sender(df,sender_name):
    
    if sender_name != 'all' :
        df=df[df['sender_name']==sender_name]        
        
        
    day_max_message=df['date_time'].dt.date.value_counts().head(2)
    
    hours=df[['content','hour']].groupby(by='hour').count().T/360
    
    day=df[['content','weekday']].groupby(by='weekday').count().T/50
    
    word_max_freq=df['parsed_content'].value_counts().head(2)
    
    
    
    return(day_max_message,hours,day,word_max_freq)


In [110]:
#messages with Jack are saved in two differnt files, going to load and clean both files seperatley and then concat them

df1 = load_all_messages('message_1.json')
df1 = clean_data(df1)

df2 = load_all_messages('message_2.json')
df2 = clean_data(df2)

df = pd.concat([df1, df2])

#with claire

claire = load_all_messages('claire.json')
claire = clean_data(claire)

#friends group chat 

pubg = load_all_messages('pubg.json')
pubg = clean_data(pubg)


In [136]:
df = add_information(df)
df_grouped=groupby_sender(df)

claire = add_information(claire)
claire_grouped = groupby_sender(claire)

pubg = add_information(pubg)
pubg_grouped = groupby_sender(pubg)

display(pubg_grouped)

C:\Users\Shipt\AppData\Local\Temp\ipykernel_17612\3076207597.py:10: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (2 levels on the left, 1 on the right)
  df_grouped=df_grouped.merge(df_grouped_has_reacfrom,left_index=True, right_index=True)
C:\Users\Shipt\AppData\Local\Temp\ipykernel_17612\3076207597.py:10: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (2 levels on the left, 1 on the right)
  df_grouped=df_grouped.merge(df_grouped_has_reacfrom,left_index=True, right_index=True)
C:\Users\Shipt\AppData\Local\Temp\ipykernel_17612\3076207597.py:10: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (2 levels on the left, 1 on the right)
  df_grouped=df_grouped.merge(df_grouped_has_reacfrom,left_index=True, right_index=True)


word_number                        is_message
                          sum  max       mean median      count
sender_name                                                    
Declan Barrett           1205  306  25.638298   16.0         47
Jason Lu                 2564  192  23.099099   20.0        111
Jes Bromley              9919  540  38.150000   27.0        260
Josh Shipton             4249  241  20.042453   14.0        212
Kartikeya Kaushal        6120  251  30.000000   24.0        204

In [118]:
#finds and shows max message

def findmaxmessage(df):
    maxValueIndex = df['word_number'].idxmax()
    maxvalue = df1.loc[maxValueIndex]
    return maxvalue['content']

In [122]:
#count certain words

to_count = ['clunge']

def wordcounter(df):
    pass


IndentationError: expected an indented block (207036395.py, line 6)

In [123]:
sender_list =  np.concatenate((df.sender_name.unique(),['all']))
for sender in sender_list : 
    day_max_message,hours,day,word_max_freq =  by_sender(df,sender)  
    
        

KeyError: 'parsed_content'